# Custom Graph Patterns - Ungraph

Este notebook demuestra cómo crear y usar patrones personalizados de grafo desde la ingesta de datos hasta la exploración del grafo.

## Objetivos

1. **Crear patrones personalizados** de estructura de grafo ✅
2. **Ingerir datos** usando patrones personalizados ✅ **IMPLEMENTADO**
3. **Explorar el grafo** creado con diferentes estructuras
4. **Comparar patrones** y sus ventajas/desventajas

## Flujo

1. Definir patrones personalizados ✅
2. Validar patrones ✅
3. Generar queries Cypher ✅
4. Ingerir datos con patrones personalizados ✅ **DISPONIBLE AHORA**
5. Explorar el grafo creado

**Nota:** La funcionalidad de ingesta con patrones personalizados está **implementada y funcional** en `ungraph.ingest_document(pattern=...)`.


In [1]:
def add_src_to_path(path_folder: str):
    ''' 
    Helper function for adding the "path_folder" directory to the path.
    in order to work on notebooks and scripts
    '''
    import sys
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            # Agregar el directorio padre para que sea un paquete Python
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
                print(f"Path Folder parent added: {parent_dir}")
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
                print(f"Path Folder {path_folder} added: {candidate}")
            return
    print(f"Not found '{path_folder}' folder on the hierarchy of directories")

# Agregar carpetas necesarias al path
add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")


Path Folder parent added: D:\projects\Ungraph
Path Folder src added: D:\projects\Ungraph\src
Path Folder src/utils added: D:\projects\Ungraph\src\utils
Path Folder src/data added: D:\projects\Ungraph\src\data


In [2]:
# Importar librerías necesarias
import sys
from pathlib import Path
from typing import Dict, Any, List

# Importar handlers desde utils
from src.utils.handlers import find_in_project

# Importar ungraph
try:
    import ungraph
    print("✅ Ungraph importado como paquete instalado")
except ImportError:
    import src
    ungraph = src
    print("✅ Ungraph importado desde src/ (modo desarrollo)")

# Importar Value Objects y servicios
from domain.value_objects.graph_pattern import (
    GraphPattern,
    NodeDefinition,
    RelationshipDefinition
)
from domain.value_objects.predefined_patterns import FILE_PAGE_CHUNK_PATTERN
from infrastructure.services.neo4j_pattern_service import Neo4jPatternService

print(f"📦 Ungraph version: {ungraph.__version__}")


d:\projects\Ungraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Ungraph importado desde src/ (modo desarrollo)
📦 Ungraph version: 0.1.0


## Parte 1: Patrón Simple - Solo Chunks

Creamos un patrón minimalista que solo almacena chunks sin estructura File-Page.


In [3]:
# Patrón 1: Solo Chunks (sin File/Page)
print("📝 Creando patrón: SIMPLE_CHUNK")
print("=" * 60)

simple_chunk_node = NodeDefinition(
    label="Chunk",
    required_properties={
        "chunk_id": str,
        "content": str,
        "embeddings": list,
        "embeddings_dimensions": int
    },
    optional_properties={
        "chunk_id_consecutive": int,
        "source_file": str
    },
    indexes=["chunk_id", "chunk_id_consecutive"]
)

SIMPLE_CHUNK_PATTERN = GraphPattern(
    name="SIMPLE_CHUNK",
    description="Solo chunks, sin estructura File-Page. Útil para documentos simples.",
    node_definitions=[simple_chunk_node],
    relationship_definitions=[],
    search_patterns=["basic", "hybrid"]
)

print(f"✅ Patrón creado:")
print(f"   Nombre: {SIMPLE_CHUNK_PATTERN.name}")
print(f"   Descripción: {SIMPLE_CHUNK_PATTERN.description}")
print(f"   Nodos: {[n.label for n in SIMPLE_CHUNK_PATTERN.node_definitions]}")
print(f"   Relaciones: {len(SIMPLE_CHUNK_PATTERN.relationship_definitions)}")


📝 Creando patrón: SIMPLE_CHUNK
✅ Patrón creado:
   Nombre: SIMPLE_CHUNK
   Descripción: Solo chunks, sin estructura File-Page. Útil para documentos simples.
   Nodos: ['Chunk']
   Relaciones: 0


In [4]:
# Validar y generar query Cypher para SIMPLE_CHUNK
service = Neo4jPatternService()

# Validar
is_valid = service.validate_pattern(SIMPLE_CHUNK_PATTERN)
print(f"✅ Validación: {'VÁLIDO' if is_valid else 'INVÁLIDO'}")

# Generar query Cypher
cypher_query = service.generate_cypher(SIMPLE_CHUNK_PATTERN, "create")
print(f"\n📝 Query Cypher generado:")
print(cypher_query)


✅ Validación: VÁLIDO

📝 Query Cypher generado:
MERGE (n0:Chunk {chunk_id: $chunk_id, content: $content, embeddings: $embeddings, embeddings_dimensions: $embeddings_dimensions})
ON CREATE SET n0.chunk_id_consecutive = $chunk_id_consecutive, n0.source_file = $source_file


## Parte 2: Patrón con Relaciones Secuenciales

Creamos un patrón que conecta chunks consecutivos directamente.


In [5]:
# Patrón 2: Chunks con relaciones secuenciales
print("📝 Creando patrón: SEQUENTIAL_CHUNKS")
print("=" * 60)

chunk_node = NodeDefinition(
    label="Chunk",
    required_properties={
        "chunk_id": str,
        "content": str,
        "embeddings": list,
        "embeddings_dimensions": int
    },
    optional_properties={
        "chunk_id_consecutive": int,
        "source_file": str
    },
    indexes=["chunk_id", "chunk_id_consecutive"]
)

# Relación: Chunk siguiente
next_chunk_rel = RelationshipDefinition(
    from_node="Chunk",
    to_node="Chunk",
    relationship_type="NEXT_CHUNK",
    direction="OUTGOING"
)

SEQUENTIAL_CHUNKS_PATTERN = GraphPattern(
    name="SEQUENTIAL_CHUNKS",
    description="Chunks con relaciones NEXT_CHUNK entre consecutivos. Útil para mantener orden secuencial.",
    node_definitions=[chunk_node],
    relationship_definitions=[next_chunk_rel],
    search_patterns=["basic", "hybrid"]
)

print(f"✅ Patrón creado:")
print(f"   Nombre: {SEQUENTIAL_CHUNKS_PATTERN.name}")
print(f"   Relaciones: {[r.relationship_type for r in SEQUENTIAL_CHUNKS_PATTERN.relationship_definitions]}")

# Validar y generar query
is_valid = service.validate_pattern(SEQUENTIAL_CHUNKS_PATTERN)
print(f"\n✅ Validación: {'VÁLIDO' if is_valid else 'INVÁLIDO'}")

cypher_query = service.generate_cypher(SEQUENTIAL_CHUNKS_PATTERN, "create")
print(f"\n📝 Query Cypher generado:")
print(cypher_query)


📝 Creando patrón: SEQUENTIAL_CHUNKS
✅ Patrón creado:
   Nombre: SEQUENTIAL_CHUNKS
   Relaciones: ['NEXT_CHUNK']

✅ Validación: VÁLIDO

📝 Query Cypher generado:
MERGE (n0:Chunk {chunk_id: $chunk_id, content: $content, embeddings: $embeddings, embeddings_dimensions: $embeddings_dimensions})
ON CREATE SET n0.chunk_id_consecutive = $chunk_id_consecutive, n0.source_file = $source_file
MERGE (n0)-[:NEXT_CHUNK]->(n0)


## Parte 3: Patrón Léxico - Entidades y Chunks

Creamos un patrón para grafo léxico con entidades extraídas y sus relaciones con chunks.


In [6]:
# Patrón 3: Grafo Léxico con Entidades
print("📝 Creando patrón: LEXICAL_GRAPH")
print("=" * 60)

# Nodo Entity
entity_node = NodeDefinition(
    label="Entity",
    required_properties={
        "name": str,
        "type": str  # "PERSON", "ORGANIZATION", "CONCEPT", etc.
    },
    optional_properties={
        "description": str,
        "frequency": int
    },
    indexes=["name", "type"]
)

# Nodo Chunk
chunk_node = NodeDefinition(
    label="Chunk",
    required_properties={
        "chunk_id": str,
        "content": str,
        "embeddings": list,
        "embeddings_dimensions": int
    },
    indexes=["chunk_id"]
)

# Relación: Chunk menciona Entity
mentions_rel = RelationshipDefinition(
    from_node="Chunk",
    to_node="Entity",
    relationship_type="MENTIONS",
    properties={"count": int},  # Número de veces que se menciona
    direction="OUTGOING"
)

# Relación: Entity relacionada con otra Entity
related_rel = RelationshipDefinition(
    from_node="Entity",
    to_node="Entity",
    relationship_type="RELATED_TO",
    properties={"strength": float},  # Fuerza de la relación
    direction="OUTGOING"
)

LEXICAL_GRAPH_PATTERN = GraphPattern(
    name="LEXICAL_GRAPH",
    description="Grafo léxico con entidades extraídas y sus relaciones. Útil para análisis semántico.",
    node_definitions=[entity_node, chunk_node],
    relationship_definitions=[mentions_rel, related_rel],
    search_patterns=["basic", "hybrid", "pattern_matching"]
)

print(f"✅ Patrón creado:")
print(f"   Nombre: {LEXICAL_GRAPH_PATTERN.name}")
print(f"   Nodos: {[n.label for n in LEXICAL_GRAPH_PATTERN.node_definitions]}")
print(f"   Relaciones: {[r.relationship_type for r in LEXICAL_GRAPH_PATTERN.relationship_definitions]}")

# Validar
is_valid = service.validate_pattern(LEXICAL_GRAPH_PATTERN)
print(f"\n✅ Validación: {'VÁLIDO' if is_valid else 'INVÁLIDO'}")

# Generar query
cypher_query = service.generate_cypher(LEXICAL_GRAPH_PATTERN, "create")
print(f"\n📝 Query Cypher generado:")
print(cypher_query)


📝 Creando patrón: LEXICAL_GRAPH
✅ Patrón creado:
   Nombre: LEXICAL_GRAPH
   Nodos: ['Entity', 'Chunk']
   Relaciones: ['MENTIONS', 'RELATED_TO']

✅ Validación: VÁLIDO

📝 Query Cypher generado:
MERGE (n0:Entity {name: $name, type: $type})
ON CREATE SET n0.description = $description, n0.frequency = $frequency
MERGE (n1:Chunk {chunk_id: $chunk_id, content: $content, embeddings: $embeddings, embeddings_dimensions: $embeddings_dimensions})
MERGE (n1)-[:MENTIONS {count: $count}]->(n0)
MERGE (n0)-[:RELATED_TO {strength: $strength}]->(n0)


In [7]:
# Patrón 4: Documento → Sección → Párrafo
print("📝 Creando patrón: DOCUMENT_SECTION_PARAGRAPH")
print("=" * 60)

document_node = NodeDefinition(
    label="Document",
    required_properties={"doc_id": str, "title": str},
    optional_properties={"created_at": int, "author": str},
    indexes=["doc_id"]
)

section_node = NodeDefinition(
    label="Section",
    required_properties={"section_id": str, "title": str, "order": int},
    optional_properties={"summary": str},
    indexes=["section_id"]
)

paragraph_node = NodeDefinition(
    label="Paragraph",
    required_properties={
        "para_id": str,
        "content": str,
        "embeddings": list,
        "embeddings_dimensions": int
    },
    optional_properties={"order": int},
    indexes=["para_id"]
)

# Relaciones
has_section = RelationshipDefinition(
    from_node="Document",
    to_node="Section",
    relationship_type="HAS_SECTION",
    direction="OUTGOING"
)

has_paragraph = RelationshipDefinition(
    from_node="Section",
    to_node="Paragraph",
    relationship_type="HAS_PARAGRAPH",
    direction="OUTGOING"
)

next_paragraph = RelationshipDefinition(
    from_node="Paragraph",
    to_node="Paragraph",
    relationship_type="NEXT_PARAGRAPH",
    direction="OUTGOING"
)

DOCUMENT_SECTION_PARAGRAPH_PATTERN = GraphPattern(
    name="DOCUMENT_SECTION_PARAGRAPH",
    description="Documento contiene secciones, secciones contienen párrafos. Alternativa jerárquica a FILE_PAGE_CHUNK.",
    node_definitions=[document_node, section_node, paragraph_node],
    relationship_definitions=[has_section, has_paragraph, next_paragraph],
    search_patterns=["basic", "hybrid", "parent_child"]
)

print(f"✅ Patrón creado:")
print(f"   Nombre: {DOCUMENT_SECTION_PARAGRAPH_PATTERN.name}")
print(f"   Nodos: {[n.label for n in DOCUMENT_SECTION_PARAGRAPH_PATTERN.node_definitions]}")
print(f"   Relaciones: {[r.relationship_type for r in DOCUMENT_SECTION_PARAGRAPH_PATTERN.relationship_definitions]}")

# Validar
is_valid = service.validate_pattern(DOCUMENT_SECTION_PARAGRAPH_PATTERN)
print(f"\n✅ Validación: {'VÁLIDO' if is_valid else 'INVÁLIDO'}")

# Generar query
cypher_query = service.generate_cypher(DOCUMENT_SECTION_PARAGRAPH_PATTERN, "create")
print(f"\n📝 Query Cypher generado:")
print(cypher_query[:500] + "..." if len(cypher_query) > 500 else cypher_query)


📝 Creando patrón: DOCUMENT_SECTION_PARAGRAPH
✅ Patrón creado:
   Nombre: DOCUMENT_SECTION_PARAGRAPH
   Nodos: ['Document', 'Section', 'Paragraph']
   Relaciones: ['HAS_SECTION', 'HAS_PARAGRAPH', 'NEXT_PARAGRAPH']

✅ Validación: VÁLIDO

📝 Query Cypher generado:
MERGE (n0:Document {doc_id: $doc_id, title: $title})
ON CREATE SET n0.created_at = $created_at, n0.author = $author
MERGE (n1:Section {section_id: $section_id, title: $title, order: $order})
ON CREATE SET n1.summary = $summary
MERGE (n2:Paragraph {para_id: $para_id, content: $content, embeddings: $embeddings, embeddings_dimensions: $embeddings_dimensions})
ON CREATE SET n2.order = $order
MERGE (n0)-[:HAS_SECTION]->(n1)
MERGE (n1)-[:HAS_PARAGRAPH]->(n2)
MERGE (n2)-[:NEXT_PARAGRAPH]->(n2)


## Parte 5: Comparar Patrones

Comparamos los diferentes patrones creados y sus características.


In [8]:
# Comparar todos los patrones
print("📊 COMPARACIÓN DE PATRONES")
print("=" * 80)

patterns = {
    "FILE_PAGE_CHUNK": FILE_PAGE_CHUNK_PATTERN,
    "SIMPLE_CHUNK": SIMPLE_CHUNK_PATTERN,
    "SEQUENTIAL_CHUNKS": SEQUENTIAL_CHUNKS_PATTERN,
    "LEXICAL_GRAPH": LEXICAL_GRAPH_PATTERN,
    "DOCUMENT_SECTION_PARAGRAPH": DOCUMENT_SECTION_PARAGRAPH_PATTERN
}

comparison_data = []
for name, pattern in patterns.items():
    comparison_data.append({
        "Nombre": name,
        "Nodos": len(pattern.node_definitions),
        "Relaciones": len(pattern.relationship_definitions),
        "Complejidad": "Alta" if len(pattern.node_definitions) > 2 else "Media" if len(pattern.node_definitions) == 2 else "Baja",
        "Uso": pattern.description[:50] + "..."
    })

# Mostrar comparación
for data in comparison_data:
    print(f"\n{data['Nombre']}:")
    print(f"  Nodos: {data['Nodos']}")
    print(f"  Relaciones: {data['Relaciones']}")
    print(f"  Complejidad: {data['Complejidad']}")
    print(f"  Uso: {data['Uso']}")


📊 COMPARACIÓN DE PATRONES

FILE_PAGE_CHUNK:
  Nodos: 3
  Relaciones: 3
  Complejidad: Alta
  Uso: Patrón básico: File contiene Pages, Pages contiene...

SIMPLE_CHUNK:
  Nodos: 1
  Relaciones: 0
  Complejidad: Baja
  Uso: Solo chunks, sin estructura File-Page. Útil para d...

SEQUENTIAL_CHUNKS:
  Nodos: 1
  Relaciones: 1
  Complejidad: Baja
  Uso: Chunks con relaciones NEXT_CHUNK entre consecutivo...

LEXICAL_GRAPH:
  Nodos: 2
  Relaciones: 2
  Complejidad: Media
  Uso: Grafo léxico con entidades extraídas y sus relacio...

DOCUMENT_SECTION_PARAGRAPH:
  Nodos: 3
  Relaciones: 3
  Complejidad: Alta
  Uso: Documento contiene secciones, secciones contienen ...


## Parte 6: Explorar Queries Cypher Generados

Examinamos los queries Cypher generados para cada patrón.


In [9]:
# Generar y comparar queries Cypher para cada patrón
print("📝 QUERIES CYPHER GENERADOS")
print("=" * 80)

for name, pattern in patterns.items():
    print(f"\n{'=' * 80}")
    print(f"Patrón: {name}")
    print(f"{'=' * 80}")
    
    try:
        cypher = service.generate_cypher(pattern, "create")
        print(cypher)
        print(f"\n✅ Query generado exitosamente ({len(cypher)} caracteres)")
    except Exception as e:
        print(f"❌ Error al generar query: {e}")


📝 QUERIES CYPHER GENERADOS

Patrón: FILE_PAGE_CHUNK
MERGE (n0:File {filename: $filename})
ON CREATE SET n0.createdAt = $createdAt
MERGE (n1:Page {filename: $filename, page_number: $page_number})
MERGE (n2:Chunk {chunk_id: $chunk_id, page_content: $page_content, embeddings: $embeddings, embeddings_dimensions: $embeddings_dimensions})
ON CREATE SET n2.is_unitary = $is_unitary, n2.chunk_id_consecutive = $chunk_id_consecutive, n2.embedding_encoder_info = $embedding_encoder_info
MERGE (n0)-[:CONTAINS]->(n1)
MERGE (n1)-[:HAS_CHUNK]->(n2)
MERGE (n2)-[:NEXT_CHUNK]->(n2)

✅ Query generado exitosamente (516 caracteres)

Patrón: SIMPLE_CHUNK
MERGE (n0:Chunk {chunk_id: $chunk_id, content: $content, embeddings: $embeddings, embeddings_dimensions: $embeddings_dimensions})
ON CREATE SET n0.chunk_id_consecutive = $chunk_id_consecutive, n0.source_file = $source_file

✅ Query generado exitosamente (222 caracteres)

Patrón: SEQUENTIAL_CHUNKS
MERGE (n0:Chunk {chunk_id: $chunk_id, content: $content, embedd

## Parte 7: Uso con Datos Reales ✅ IMPLEMENTADO

**Nota:** La integración de patrones personalizados con `ingest_document()` está **implementada y funcional** (Fase 2 completa).

Ahora puedes:
1. Crear y validar patrones ✅
2. Generar queries Cypher ✅
3. **Ingerir datos con patrones personalizados** ✅ **DISPONIBLE AHORA**

**Ejemplo de uso:**

```python
# Ingerir documento con patrón personalizado
chunks = ungraph.ingest_document(
    "documento.md",
    pattern=SIMPLE_CHUNK_PATTERN  # Usar patrón personalizado
)
```


In [10]:
# Probar ingesta con patrones personalizados
print("📝 PROBAR INGESTA CON PATRONES PERSONALIZADOS")
print("=" * 60)

# Encontrar archivos de datos
data_path = find_in_project(
    target="data",
    search_type="folder",
    project_root=None
)

if data_path:
    print(f"✅ Carpeta de datos encontrada: {data_path}")
    files = list(data_path.glob("*"))
    print(f"\n📄 Archivos disponibles ({len([f for f in files if f.is_file()])}):")
    for file in files:
        if file.is_file():
            print(f"   - {file.name}")
    
    print("\n💡 Ejemplo de uso con patrón personalizado:")
    print("   chunks = ungraph.ingest_document(file_path, pattern=SIMPLE_CHUNK_PATTERN)")
    print("\n⚠️  Descomenta el código siguiente para probar con datos reales:")
    
    # Ejemplo comentado para probar
    """
    # Seleccionar un archivo de prueba
    test_file = list(data_path.glob("*.md"))[0] if list(data_path.glob("*.md")) else None
    
    if test_file:
        print(f"\n🔍 Probando ingesta con patrón SIMPLE_CHUNK...")
        try:
            chunks = ungraph.ingest_document(
                str(test_file),
                pattern=SIMPLE_CHUNK_PATTERN,
                chunk_size=500,
                chunk_overlap=100
            )
            print(f"✅ Ingesta exitosa: {len(chunks)} chunks creados")
            print(f"   Archivo: {test_file.name}")
            print(f"   Patrón usado: {SIMPLE_CHUNK_PATTERN.name}")
        except Exception as e:
            print(f"❌ Error: {e}")
            print("💡 Asegúrate de tener Neo4j configurado y corriendo")
    """
else:
    print("⚠️  Carpeta de datos no encontrada")


📝 PROBAR INGESTA CON PATRONES PERSONALIZADOS
✅ Carpeta de datos encontrada: D:\projects\Ungraph\src\data

📄 Archivos disponibles (3):
   - 110225.md
   - AnnyLetter.txt
   - Usar símboles de silencio de corchea.docx

💡 Ejemplo de uso con patrón personalizado:
   chunks = ungraph.ingest_document(file_path, pattern=SIMPLE_CHUNK_PATTERN)

⚠️  Descomenta el código siguiente para probar con datos reales:


## Parte 8: Resumen

Resumen de patrones creados y próximos pasos.


In [ ]:
print("\n" + "=" * 80)
print("📊 RESUMEN DE PATRONES CREADOS")
print("=" * 80)

summary = {
    "Patrones Creados": len(patterns),
    "Patrones Validados": sum(1 for p in patterns.values() if service.validate_pattern(p)),
    "Queries Generados": len([p for p in patterns.values() if service.generate_cypher(p, "create")])
}

for key, value in summary.items():
    print(f"{key}: {value}")

print("\n📋 Patrones disponibles:")
for name in patterns.keys():
    print(f"  ✅ {name}")

print("\n🎯 Estado de Implementación:")
print("  ✅ Fase 1: Fundación - COMPLETA")
print("  ✅ Fase 2: Integración con Ingesta - COMPLETA")
print("  ✅ Fase 3: Búsqueda GraphRAG Básicos - COMPLETA")
print("\n🎯 Próximos Pasos:")
print("  1. ⏳ Probar patrones con datos reales")
print("  2. ⏸️  Fase 3 Avanzada: Implementar patrones avanzados (si hay demanda)")
print("  3. ⏸️  Optimización y mejoras")
print("=" * 80)



📊 RESUMEN DE PATRONES CREADOS
Patrones Creados: 5
Patrones Validados: 5
Queries Generados: 5

📋 Patrones disponibles:
  ✅ FILE_PAGE_CHUNK
  ✅ SIMPLE_CHUNK
  ✅ SEQUENTIAL_CHUNKS
  ✅ LEXICAL_GRAPH
  ✅ DOCUMENT_SECTION_PARAGRAPH

🎯 Estado de Implementación:
  ✅ Fase 1: Fundación - COMPLETA
  ✅ Fase 2: Integración con Ingesta - COMPLETA
  ✅ Fase 3: Búsqueda GraphRAG Básicos - COMPLETA

🎯 Próximos Pasos:
  1. ⏳ Probar patrones con datos reales
  2. ⏸️  Fase 3 Avanzada: Implementar patrones avanzados (si hay demanda)
  3. ⏸️  Optimización y mejoras


: 

## Referencias

- [Documentación de Patrones](../../docs/concepts/graph-patterns.md)
- [Guía de Patrones Personalizados](../../docs/guides/custom-patterns.md)
- [Plan de Patrones de Grafo](../../_PLAN_PATRONES_GRAFO.md)
- [GraphRAG Pattern Catalog](https://graphrag.com/reference/)
